# MMVC Trainer

ver.2022/12/19

「Google Colaboratory」を利用してMMVCで利用するVITSの学習を行います。

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer" #@param {type:"string"}

%cd $directory
!ls -1

​

In [ ]:
#@title ## 3 GPUの確認
#@markdown GPUの確認を行います。  
#@markdown 割り当てられたGPUのメモリーを確認し、それに合わせてconfigファイルの"batch_size"を指定してください。  
!nvidia-smi

​

In [ ]:
#@title ## 4 ライブラリのインストール
#@markdown 時間がかかります。気長にお待ちください。
!apt-get install espeak
!pip install -r requirements.txt

%cd monotonic_align/
!python setup.py build_ext --inplace
%cd ../

​

In [ ]:
#@title ## 5 tensorboardの起動
#@markdown とりあえず実行しておいてください。
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs --port 5000

​

In [ ]:
#@title ## 6 学習を実行する
#@markdown 学習を実行します。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown ### Settings
#@markdown New / Resume

#@markdown 新規に学習を開始する場合は、Newを選択し、**以下の-c, -m, -fg, -fdを全て設定してください。**

#@markdown 学習を再開する場合は、Resumeを選択し、以下の-c, -mを設定してください。-mで指定したディレクトリの最新のモデルから学習を再開します。 

#@markdown **注意**

#@markdown **学習を再開する前に、"1. Google Driveをマウント"から"5. tensorboardの起動"までを実行してください。ノートブックを開くたびに、毎回必要です。**

New_or_Resume = "New" #@param ["New", "Resume"]

#@markdown -c：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。
config_pass = "configs/train_config.json" #@param {type:"string"}

#@markdown -m：modelの保存先ディレクトリ  
#@markdown **ディレクトリ名を直接指名してください。** 
model_save_dic = "20220306_24000" #@param {type:"string"}

#@markdown -fg：(新規学習時のみ) Fine tuningのベースとなるG_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。
fine_model_g = "fine_model/G_180000.pth" #@param {type:"string"}

#@markdown -fd：(新規学習時のみ) Fine tuningのベースとなるD_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。
fine_model_d = "fine_model/D_180000.pth" #@param {type:"string"}

if New_or_Resume == "New":
  !python train_ms.py -c $config_pass -m $model_save_dic -fg $fine_model_g -fd $fine_model_d
elif New_or_Resume == "Resume":
  !python train_ms.py -c $config_pass -m $model_save_dic

In [ ]:
#@title ## 7 学習結果をONNXファイルに変換

#@markdown 学習結果をONNXファイルに変換します。

!python onnx_export.py --config_file logs/$model_save_dic/config.json --convert_pth logs/$model_save_dic/G_latest_99999999.pth

#@markdown 変換が終わるとlogs内にあるmodelの保存先ディレクトリにONNXファイルが生成されています。  

#@markdown クライアントの実行には、このディレクトリ内の **config.json** と **G_latest_99999999.onnx** の2ファイルが必要です。  

#@markdown クライアントのlogsディレクトリ内に、わかりやすいように同じ名前でフォルダを作成し、この2ファイルをコピーしてください。

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## 1. サポートセッションの開始

#@markdown このセルを実行すると、このノートブックでの出力が全て記録されるようになります。この間、**ノートブック上には出力が表示されません**が、仕様ですのでご安心ください。

#@markdown 問題が再現できたら、以下の「2. サポートセッションの終了」を実行してください。

#@markdown なお、誤って実行してしまった場合は、一度ランタイムを切断し、再接続してください。

#ディレクトリ内の作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
ｊst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#stdoutのキャプチャ
import sys
class Logger:

    def __init__(self, path):
        self.file = open(path, 'w')
 
    def write(self, message):
        self.file.write(message)

    def flush(self):
        self.file.flush()
 
path = "/mmvc-debug/mmvc-" + str(nowt) + "-stdout.txt"
sys.stdout = Logger(path)

​

In [ ]:
#@title ## 2. サポートセッションの終了

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数(自動的に設定されるものと、ユーザーが入力するものがあります)のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    #変数名(str)
    print(ev, end=' : ', file=f) 
    #変数の型(変数名がstrとなっているためevalでkeyに直す)
    print(type(eval(ev)), end=' : ', file=f)
    #変数の内容(変数名がstrとなっているためevalでkeyに直す)
    print(eval(ev), file=f)

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {path} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)